<a href="https://colab.research.google.com/github/lx-jdar/progra-concurrente/blob/development/tp1/go/tp1_goland.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabajo Práctico 1 - Parte 1: GOLANG





Instalacion de golang en Colab

In [ ]:
!add-apt-repository ppa:longsleep/golang-backports -y
!apt update
!apt install golang-go
%env GOPATH=/root/go
!go version

Creación del Archivo GO

In [2]:
%%writefile tp1-golang.go
package main

import (
	"fmt"
	"os"
	"syscall"
	"time"
)

var nodes map[string]string = map[string]string{
	"A": "CB", "B": "ED", "C": "F",
	"D": "", "E": "HG", "F": ""}

func fork() (uintptr, error) {
	// fork off the parent process
	ret, ret2, errno := syscall.RawSyscall(syscall.SYS_FORK, 0, 0, 0)
	// failure while forking
	if errno != 0 {
		return uintptr(errno), fmt.Errorf("fork failed with error %s", errno.Error())
	}

	// failure
	if ret2 < 0 {
		return ret2, fmt.Errorf("fork failed with pid %d", ret2)
	}

	return ret, nil
}

func createProcesses(letter string) {
	if letter != "A" {
		fmt.Printf("PID_Child: %5d Padre: %5d Node: %s\n", os.Getpid(), os.Getppid(), letter)
	}
	children := make([]uintptr, len(nodes[letter]))
	var node string
	for i := 0; i < len(nodes[letter]); i++ {
		node = string(nodes[letter][i])
		children[i], _ = fork()
		if children[i] == 0 {
			createProcesses(node)
			return
		}
	}
	for child := range children {
		syscall.Wait4(int(child), nil, 0, nil)
	}
	time.Sleep(10 * time.Second)
	return
}

func main() {
	fmt.Printf("PPID_INIT: %5d Padre: %5d Node: A\n", os.Getpid(), os.Getppid())
	createProcesses("A")
}

Writing tp1-golang.go


Compilacion del Archivo GO

In [3]:
!go build -o tp1Golang tp1-golang.go

Ejecución del Programa

In [ ]:
!./tp1Golang

Ejecucion del Programa en background logueando a Archivo

In [10]:
!nohup ./tp1Golang 1>salidaGO 2>/dev/null &

Visualizacion de los Procesos

In [11]:
!ps -axf | grep tp1Golang

   3447 ?        S      0:00  |       \_ /bin/bash -c ps -axf | grep tp1Golang
   3449 ?        S      0:00  |           \_ grep tp1Golang
   3420 ?        Sl     0:00 ./tp1Golang
   3424 ?        S      0:00  \_ ./tp1Golang
   3426 ?        S      0:00  |   \_ ./tp1Golang
   3425 ?        S      0:00  \_ ./tp1Golang
   3427 ?        S      0:00      \_ ./tp1Golang
   3429 ?        S      0:00      |   \_ ./tp1Golang
   3430 ?        S      0:00      |   \_ ./tp1Golang
   3428 ?        S      0:00      \_ ./tp1Golang


Visualizacion de log del Programa GO a un Archivo

In [12]:
!cat salidaGO && echo

PPID_INIT:  3420 Padre:     1 Node: A
PID_Child:  3424 Padre:  3420 Node: C
PID_Child:  3425 Padre:  3420 Node: B
PID_Child:  3426 Padre:  3424 Node: F
PID_Child:  3427 Padre:  3425 Node: E
PID_Child:  3428 Padre:  3425 Node: D
PID_Child:  3429 Padre:  3427 Node: H
PID_Child:  3430 Padre:  3427 Node: G



# Comentarios
> En el caso de Golang, en primera en instancia cabe decir que no hay mucha documentacion respecto a como manejar procesos pesados, la concurrencia en Go en cuanto a procesos parece buena, pero se debe realizar muchas pruebas hasta entender como funciona el manejo de procesos. Hubo que buscar información, interiorizarse en la instalación y la sintaxis del lenguaje apropaido de declaración de variables, etc. Luego de una primera aproximación, se vuelve mas fácil interactuar entre procesos.
GO tiene varias formas de hacer forks, que dan una mayor amplitud de organización en la logica.

# Conclusión
> Go es una lenguaje que se va adentrando cada vez mas en el ambito de desarrollo ya que implementa nuevas formas de definiciones. Ademas incorpora alguna de las mejores practicas provenientes de otros lenguajes, entre ellos c/c++, python y java como así otras nuevas. Hace uso de un lenguaje mas simplificado, con lo que una vez en el tema, su escalamiento puede ser mayor en comparación a otros lenguajes. Puede ser interpretado y compilado. Este último resulta en una alta performance.